In [1]:
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import ElementNotInteractableException, ElementClickInterceptedException
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
import pandas as pd
import time


def get_card_details(driver, benefit_name):
    card_info = {}

    # 혜택 이름 저장
    card_info['혜택 이름'] = benefit_name

    # 카드 이름 크롤링
    card_name = driver.find_element(By.CSS_SELECTOR, '.tit .card').text
    card_info['카드이름'] = card_name

    # 주요 혜택 정보 크롤링
    benefits = driver.find_elements(By.CSS_SELECTOR, '.bnf1 dl')
    card_benefits = {}
    for benefit in benefits:
        key = benefit.find_element(By.CSS_SELECTOR, 'dt').text
        value = benefit.find_element(By.CSS_SELECTOR, 'dd strong').text
        card_benefits[key] = value
    card_info['주요 혜택'] = card_benefits

    # 일반 혜택 정보 크롤링
    general_benefits = driver.find_elements(By.CSS_SELECTOR, '.lst.bene_area dl')
    general_benefit_list = []
    for benefit in general_benefits:
        title = benefit.find_element(By.CSS_SELECTOR, 'dt .txt1').text
        description = benefit.find_element(By.CSS_SELECTOR, 'dt i').text
        general_benefit_list.append(f"{title}: {description}")
    card_info['혜택'] = general_benefit_list

    # 연회비 정보 크롤링
    annual_fee_info = driver.find_element(By.CSS_SELECTOR, '.bnf2 dl:nth-child(1) dd.in_out').text
    card_info['연회비'] = annual_fee_info

    # 전월실적 정보 크롤링
    last_month_performance = driver.find_element(By.CSS_SELECTOR, '.bnf2 dl:nth-child(2) dd').text
    card_info['전월실적'] = last_month_performance

    return card_info

# 웹드라이버 시작
service = Service('./chromedriver')
driver = webdriver.Chrome(service=service)

# URL 접속
URL = "https://www.card-gorilla.com/team/detail/9"
driver.get(URL)

# 혜택별 추천 카드 목록을 찾습니다.
benefit_links = WebDriverWait(driver, 10).until(
    EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".benefit_box.team_9 a"))
)

all_card_info = []

for idx in range(len(benefit_links)):
    # 매번 새로운 혜택 링크 참조를 얻습니다.
    benefit_link = WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".benefit_box.team_9 a"))
    )[idx]

    benefit_name = benefit_link.text  # 혜택 이름 가져오기
    benefit_link.click()
    time.sleep(2)
    
    crawled_cards = 0  # 크롤링한 카드의 수를 저장하는 변수

    while True:
        # '자세히 보기' 링크들을 찾습니다.
        detail_links = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, '.card-container .benefit .b_view'))
        )

        # 이전에 크롤링한 카드의 수만큼 링크를 건너뜁니다.
        for i in range(crawled_cards, len(detail_links)):
            detail_link = WebDriverWait(driver, 10).until(
                EC.presence_of_all_elements_located((By.CSS_SELECTOR, '.card-container .benefit .b_view'))
            )[i]
            driver.execute_script("arguments[0].click();", detail_link)
            time.sleep(2)

            card_info = get_card_details(driver, benefit_name)
            all_card_info.append(card_info)
            print(f"{card_info['카드이름']} 정보를 가져왔습니다.")

            driver.back()
            time.sleep(2)
            crawled_cards += 1

        try:
            more_button = WebDriverWait(driver, 3).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "a.lst_more"))
            )
            
            # 스크롤을 아래로 이동시키기
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(2)
            
            # 카드 더보기 클릭
            ActionChains(driver).move_to_element(more_button).click().perform()
            time.sleep(2)
            
        except (TimeoutException, ElementNotInteractableException):
            # '카드 더보기' 버튼이 없으면 반복을 종료합니다.
            break

    # 원래의 혜택 페이지로 돌아옵니다.
    driver.get(URL)
    time.sleep(2)

    # 혜택 링크를 다시 가져옵니다.
    benefit_links = WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".benefit_box.team_9 a"))
    )
    
driver.close()

# 결과를 DataFrame으로 변환 후 엑셀 파일로 저장
df = pd.DataFrame(all_card_info)
df.to_excel('card_info.xlsx', index=False)

print("엑셀 파일 저장 완료!")




KeyboardInterrupt

